In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("C:/Users/ishan/Desktop/ISHANAY/BU docs/Spring 2025/Financial_analytics/Project/FinancialScorePredictor_UsingSentimentAnalysis/data/technical_analysis_wrds.csv")
df["date"] = pd.to_datetime(df["date"])
df["TICKER"] = df["TICKER"].str.upper()
df["PRC"] = pd.to_numeric(df["PRC"], errors='coerce').abs()
df["CFACPR"] = pd.to_numeric(df["CFACPR"], errors='coerce')

C:\Users\ishan\AppData\Local\Temp\ipykernel_19816\3754010988.py:4: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/ishan/Desktop/ISHANAY/BU docs/Spring 2025/Financial_analytics/Project/FinancialScorePredictor_UsingSentimentAnalysis/data/technical_analysis_wrds.csv")


In [2]:
df

,PERMNO,date,NCUSIP,TICKER,COMNAM,PERMCO,CUSIP,PRC,VOL,RET,CFACPR
0,10104,2021-12-27,68389X10,ORCL,ORACLE CORP,8045,68389X10,89.54000,7668988.0,-0.002006,1.0
1,10104,2021-12-28,68389X10,ORCL,ORACLE CORP,8045,68389X10,88.65000,5614472.0,-0.00994,1.0
2,10104,2021-12-29,68389X10,ORCL,ORACLE CORP,8045,68389X10,88.21000,5628531.0,-0.004963,1.0
3,10104,2021-12-30,68389X10,ORCL,ORACLE CORP,8045,68389X10,88.01000,7382356.0,-0.002267,1.0
4,10104,2021-12-31,68389X10,ORCL,ORACLE CORP,8045,68389X10,87.21000,5908158.0,-0.00909,1.0
...,...,...,...,...,...,...,...,...,...,...,...
348594,93436,2024-08-29,88160R10,TSLA,TESLA INC,53453,88160R10,206.28000,62056053.0,0.002576,1.0
348595,93436,2024-08-30,88160R10,TSLA,TESLA INC,53453,88160R10,214.11000,63024686.0,0.037958,1.0
348596,93436,2024-09-03,88160R10,TSLA,TESLA INC,53453,88160R10,210.60001,76310864.0,-0.016393,1.0
348597,93436,2024-09-04,88160R10,TSLA,TESLA INC,53453,88160R10,219.41000,80182824.0,0.041833,1.0


In [3]:
# 2. Adjust Prices
df["adj_price"] = df["PRC"] / df["CFACPR"]

# ---------------------------------------
# 3. Calculate Indicators by Ticker
# ---------------------------------------
df.sort_values(["TICKER", "date"], inplace=True)

def compute_tech_indicators(group):
    group = group.copy()
    group["ma_20"] = group["adj_price"].rolling(window=20).mean()
    group["ma_50"] = group["adj_price"].rolling(window=50).mean()
    group["ma_200"] = group["adj_price"].rolling(window=200).mean()
    
    # Trend signal based on MA20 vs MA50
    group["ma_signal"] = np.where(group["ma_20"] > group["ma_50"], "Uptrend", "Downtrend")

    # Bollinger Bands (20-day)
    rolling_mean = group["adj_price"].rolling(window=20).mean()
    rolling_std = group["adj_price"].rolling(window=20).std()
    group["bb_upper"] = rolling_mean + (2 * rolling_std)
    group["bb_lower"] = rolling_mean - (2 * rolling_std)

    # Bollinger signal
    group["bb_signal"] = np.where(group["adj_price"] > group["bb_upper"], "Overbought",
                           np.where(group["adj_price"] < group["bb_lower"], "Oversold", "Neutral"))
    group["daily_return"] = group["adj_price"].pct_change()
    return group

df = df.groupby("TICKER").apply(compute_tech_indicators).reset_index(drop=True)


C:\Users\ishan\AppData\Local\Temp\ipykernel_19816\764909724.py:27: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  group["daily_return"] = group["adj_price"].pct_change()
C:\Users\ishan\AppData\Local\Temp\ipykernel_19816\764909724.py:27: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  group["daily_return"] = group["adj_price"].pct_change()
C:\Users\ishan\AppData\Local\Temp\ipykernel_19816\764909724.py:27: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or spec

In [4]:
df.head()

,PERMNO,date,NCUSIP,TICKER,COMNAM,PERMCO,CUSIP,PRC,VOL,RET,CFACPR,adj_price,ma_20,ma_50,ma_200,ma_signal,bb_upper,bb_lower,bb_signal,daily_return
0,87432,2021-12-27,00846U10,A,AGILENT TECHNOLOGIES INC,36364,00846U10,158.74001,951771.0,0.005957,1.0,158.74001,NaN,NaN,NaN,Downtrend,NaN,NaN,Neutral,NaN
1,87432,2021-12-28,00846U10,A,AGILENT TECHNOLOGIES INC,36364,00846U10,159.17999,764279.0,0.002772,1.0,159.17999,NaN,NaN,NaN,Downtrend,NaN,NaN,Neutral,0.002772
2,87432,2021-12-29,00846U10,A,AGILENT TECHNOLOGIES INC,36364,00846U10,160.64999,833052.0,0.009235,1.0,160.64999,NaN,NaN,NaN,Downtrend,NaN,NaN,Neutral,0.009235
3,87432,2021-12-30,00846U10,A,AGILENT TECHNOLOGIES INC,36364,00846U10,160.88000,771020.0,0.001432,1.0,160.88000,NaN,NaN,NaN,Downtrend,NaN,NaN,Neutral,0.001432
4,87432,2021-12-31,00846U10,A,AGILENT TECHNOLOGIES INC,36364,00846U10,159.64999,1315542.0,-0.007646,1.0,159.64999,NaN,NaN,NaN,Downtrend,NaN,NaN,Neutral,-0.007646


In [5]:
# ---------------------------------------
# 4. Calculate Sharpe Ratio
# ---------------------------------------
sharpe_df = df.groupby("TICKER")["daily_return"].agg(
    lambda r: ((r.mean() - 0.042 / 252) / r.std()) * np.sqrt(252)
).reset_index(name="sharpe_ratio")

In [6]:
sharpe_df.head()

,TICKER,sharpe_ratio
0,A,-0.159643
1,AA,-0.239262
2,AAL,-0.284250
3,AAON,0.572435
4,AAPL,0.268493


In [7]:
# 5. Keep Latest Row Per Ticker
#latest = df.sort_values("date").groupby("TICKER").tail(1)
latest = df.merge(sharpe_df, on="TICKER", how="left")

In [8]:
latest

,PERMNO,date,NCUSIP,TICKER,COMNAM,PERMCO,CUSIP,PRC,VOL,RET,...,adj_price,ma_20,ma_50,ma_200,ma_signal,bb_upper,bb_lower,bb_signal,daily_return,sharpe_ratio
0,87432,2021-12-27,00846U10,A,AGILENT TECHNOLOGIES INC,36364,00846U10,158.74001,951771.0,0.005957,...,158.74001,NaN,NaN,NaN,Downtrend,NaN,NaN,Neutral,NaN,-0.159643
1,87432,2021-12-28,00846U10,A,AGILENT TECHNOLOGIES INC,36364,00846U10,159.17999,764279.0,0.002772,...,159.17999,NaN,NaN,NaN,Downtrend,NaN,NaN,Neutral,0.002772,-0.159643
2,87432,2021-12-29,00846U10,A,AGILENT TECHNOLOGIES INC,36364,00846U10,160.64999,833052.0,0.009235,...,160.64999,NaN,NaN,NaN,Downtrend,NaN,NaN,Neutral,0.009235,-0.159643
3,87432,2021-12-30,00846U10,A,AGILENT TECHNOLOGIES INC,36364,00846U10,160.88000,771020.0,0.001432,...,160.88000,NaN,NaN,NaN,Downtrend,NaN,NaN,Neutral,0.001432,-0.159643
4,87432,2021-12-31,00846U10,A,AGILENT TECHNOLOGIES INC,36364,00846U10,159.64999,1315542.0,-0.007646,...,159.64999,NaN,NaN,NaN,Downtrend,NaN,NaN,Neutral,-0.007646,-0.159643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348594,13788,2024-08-29,98978V10,ZTS,ZOETIS INC,54327,98978V10,182.89000,1359892.0,-0.000109,...,182.89000,183.294500,179.417200,179.00275,Uptrend,188.997570,177.591431,Neutral,-0.000109,-0.373673
348595,13788,2024-08-30,98978V10,ZTS,ZOETIS INC,54327,98978V10,183.49001,2328501.0,0.003281,...,183.49001,183.439501,179.723400,179.07605,Uptrend,188.998697,177.880305,Neutral,0.003281,-0.373673
348596,13788,2024-09-03,98978V10,ZTS,ZOETIS INC,54327,98978V10,183.57001,1680951.0,0.000436,...,183.57001,183.877001,179.973001,179.13065,Uptrend,187.679781,180.074221,Neutral,0.000436,-0.373673
348597,13788,2024-09-04,98978V10,ZTS,ZOETIS INC,54327,98978V10,187.33000,1870954.0,0.020483,...,187.33000,183.979002,180.282801,179.19420,Uptrend,188.041902,179.916101,Neutral,0.020483,-0.373673


In [9]:
latest.to_csv("C:/Users/ishan/Desktop/ISHANAY/BU docs/Spring 2025/Financial_analytics/Project/FinancialScorePredictor_UsingSentimentAnalysis/data/technical_indicators_wrds_output.csv", index=False)

In [3]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

# 1) Load your sentiment+fundamental file
sent_df = pd.read_csv("C:/Users/ishan/Desktop/ISHANAY/BU docs/Spring 2025/Financial_analytics/Project/FinancialScorePredictor_UsingSentimentAnalysis/data/combined_verdict_with_fundamentals.csv", parse_dates=["date"])
# Take only the most recent row per ticker
sent_latest = (
    sent_df
    .sort_values(["ticker", "date"])
    .groupby("ticker", as_index=False)
    .last()
)

# 2) Load your technical summary (Sharpe‐based) file
tech_df = pd.read_csv("C:/Users/ishan/Desktop/ISHANAY/BU docs/Spring 2025/Financial_analytics/Project/FinancialScorePredictor_UsingSentimentAnalysis/data/technical_indicators_wrds_output.csv", parse_dates=["date"])
# Rename columns to match
tech_latest = tech_df.rename(columns={"TICKER":"ticker"})

# 3) Merge on ticker
df = pd.merge(
    sent_latest,
    tech_latest[["ticker", "sharpe_ratio"]],
    on="ticker",
    how="inner"
)

# 4) Compute a normalized technical score (0–1)
df["technical_score"] = df["sharpe_ratio"] / 3.0

# 5) Compute a simple equal‐weight final_score
df["final_score"] = (
    df["final_sentiment_score"] +
    df["fundamental_score"] +
    df["technical_score"]
) / 3.0

# 6) Prepare the 1-D array for clustering
X = df["final_score"].dropna().values.reshape(-1, 1)

# 7) Run KMeans with 5 clusters
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(X)

# 8) Sort the cluster centers
centers = sorted(kmeans.cluster_centers_.flatten())
print("Cluster centers (sorted):", np.round(centers, 3))

# 9) Compute midpoints between adjacent centers as thresholds
thresholds = [
    (centers[i] + centers[i+1]) / 2 for i in range(len(centers) - 1)
]
print("Suggested thresholds (mid‐cluster):", np.round(thresholds, 3))

# Optional: assign each ticker to its cluster
df["cluster_label"] = kmeans.predict(df["final_score"].values.reshape(-1,1))
print("\nSample of tickers with their final_score and cluster:")
print(df[["ticker","final_score","cluster_label"]].sort_values("final_score").head(10))
print(df[["ticker","final_score","cluster_label"]].sort_values("final_score").tail(10))


C:\Users\ishan\AppData\Local\Temp\ipykernel_9648\2390854462.py:16: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  tech_df = pd.read_csv("C:/Users/ishan/Desktop/ISHANAY/BU docs/Spring 2025/Financial_analytics/Project/FinancialScorePredictor_UsingSentimentAnalysis/data/technical_indicators_wrds_output.csv", parse_dates=["date"])


Cluster centers (sorted): [-0.07   0.017  0.132  0.31   0.431]
Suggested thresholds (mid‐cluster): [-0.026  0.075  0.221  0.371]

Sample of tickers with their final_score and cluster:
       ticker  final_score  cluster_label
221305    PFE    -0.177705              4
221304    PFE    -0.177705              4
221303    PFE    -0.177705              4
221270    PFE    -0.177705              4
221269    PFE    -0.177705              4
221268    PFE    -0.177705              4
221267    PFE    -0.177705              4
221266    PFE    -0.177705              4
221265    PFE    -0.177705              4
221296    PFE    -0.177705              4
       ticker  final_score  cluster_label
112129   FICO     0.584137              0
112160   FICO     0.584137              0
112159   FICO     0.584137              0
112158   FICO     0.584137              0
112157   FICO     0.584137              0
112789   FICO     0.584137              0
112788   FICO     0.584137              0
112787   FICO     